In [5]:
import pandas as pd
import numpy as np
import torch
from matplotlib import pyplot as plt

import torchaudio

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix

import os

In [6]:
import splitfolders

In [7]:
from pydub import AudioSegment

# cut all audios to 3 seconds

In [1]:
for directory in ["up", "down", "left", "right"]:
    for d in os.listdir("./dataset/proccessed/" + directory):
        
        path_to_file = "/mnt/g/onlinelessons/Multimedia/HW/project/dataset/proccessed/" + directory + '/'+ d
        print(path_to_file)
        
        file = song = AudioSegment.from_wav(path_to_file)
        
        three_second = 3 * 1000
        
        cut_file = file[: three_second]
        
        cut_file.export(path_to_file,
                        format="wav")
        


### check for valid cut

In [2]:
error_flag = 0
for directory in ["up", "down", "left", "right"]:
    for d in os.listdir("./dataset/proccessed/" + directory):
        
        path_to_file = "/mnt/g/onlinelessons/Multimedia/HW/project/dataset/proccessed/" + directory + '/'+ d
        file = AudioSegment.from_file(path_to_file)
        if(file.duration_seconds != 3.0):
            print(f"found error in file: \n {path_to_file} \n duration is : {file.duration_seconds}")
            error_flag = 1

if(~error_flag):
    print("All Done")

# Down sampling to 16000

In [ ]:
import torch.nn as nn
import torch
import torchaudio


class ChangeSampleRate(nn.Module):
    def __init__(self, input_rate: int, output_rate: int):
        super().__init__()
        self.output_rate = output_rate
        self.input_rate = input_rate

    def forward(self, wav: torch.tensor) -> torch.tensor:
        # Only accepts 1-channel waveform input
        wav = wav.view(wav.size(0), -1)
        new_length = wav.size(-1) * self.output_rate // self.input_rate
        indices = (torch.arange(new_length) * (self.input_rate / self.output_rate))
        round_down = wav[:, indices.long()]
        round_up = wav[:, (indices.long() + 1).clamp(max=wav.size(-1) - 1)]
        output = round_down * (1. - indices.fmod(1.)).unsqueeze(0) + round_up * indices.fmod(1.).unsqueeze(0)
        return output


if __name__ == '__main__':
    for directory in ["up", "down", "left", "right"]:
        for d in os.listdir("./dataset/proccessed/" + directory):
            
            print("/mnt/g/onlinelessons/Multimedia/HW/project/dataset/proccessed/" + directory + '/'+ d)
            
            wav, sr = torchaudio.load("/mnt/g/onlinelessons/Multimedia/HW/project/dataset/proccessed/" + directory + '/'+ d
                                     )
            osr = 16000
            batch = wav.unsqueeze(0).repeat(10, 1, 1)
            csr = ChangeSampleRate(sr, osr)
            out_wavs = csr(wav)
            print (out_wavs)
            torchaudio.save("/mnt/g/onlinelessons/Multimedia/HW/project/dataset/proccessed/" + directory + '/'+ d, out_wavs, osr)

In [ ]:
 os.listdir("./dataset/proccessed")

In [ ]:
metadata = torchaudio.info("/mnt/g/onlinelessons/Multimedia/HW/project/dataset/proccessed/up/AnyConv.com__msg-1583385616-23.wav")
print(metadata)

# Train , Test, Validation split

In [9]:

splitfolders.ratio("./dataset/proccessed/",
                   output="./dataset/final_data/",
                   seed=1337,
                   ratio=(0.7, 0.1,0.2))

# creating csv dataset 

In [10]:
dataset_dir = "/mnt/g/onlinelessons/Multimedia/HW/project/dataset/final_data/"
os.listdir(dataset_dir)

['test', 'train', 'val']

In [11]:
def absoluteFilePaths(directory):
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            yield os .path.abspath(os.path.join(dirpath, f))

In [12]:
test_data = []
train_data = []
val_data = []
for d in absoluteFilePaths(dataset_dir):
    file_label = d.split("/")[-2]
    data_type = d.split("/")[-3]
    if data_type == "test":
        print(f"putting {d} in {data_type} group")
        
        test_data.append([d, file_label])
    elif data_type == "train":
        print(f"putting {d} in {data_type} group")
        
        train_data.append([d, file_label])
    else:
        print(f"putting {d} in {data_type} group")
        
        val_data.append([d, file_label])

columns = ["data", "label"]      

train_df = pd.DataFrame(train_data, columns = columns)
test_df = pd.DataFrame(test_data, columns = columns)
val_df = pd.DataFrame(val_data, columns = columns)

train_df.to_csv("./dataset/final_data/train_data.csv")
test_df.to_csv("./dataset/final_data/test_data.csv")
val_df.to_csv("./dataset/final_data/val_data.csv")

putting /mnt/g/onlinelessons/Multimedia/HW/project/dataset/final_data/test/down/1402-04-03 10.54.54.wav in test group
putting /mnt/g/onlinelessons/Multimedia/HW/project/dataset/final_data/test/down/1402-04-03 10.54.58.wav in test group
putting /mnt/g/onlinelessons/Multimedia/HW/project/dataset/final_data/test/down/1402-04-03 10.54.58_noise.wav in test group
putting /mnt/g/onlinelessons/Multimedia/HW/project/dataset/final_data/test/down/1402-04-03 10.55.24.wav in test group
putting /mnt/g/onlinelessons/Multimedia/HW/project/dataset/final_data/test/down/1402-04-03 10.55.24_noise.wav in test group
putting /mnt/g/onlinelessons/Multimedia/HW/project/dataset/final_data/test/down/1402-04-03 10.55.32_noise.wav in test group
putting /mnt/g/onlinelessons/Multimedia/HW/project/dataset/final_data/test/down/1402-04-03 10.55.44_noise.wav in test group
putting /mnt/g/onlinelessons/Multimedia/HW/project/dataset/final_data/test/down/1402-04-03 10.55.48.wav in test group
putting /mnt/g/onlinelessons/Mul

In [13]:
print(train_df.head())

                                                data label
0  /mnt/g/onlinelessons/Multimedia/HW/project/dat...  down
1  /mnt/g/onlinelessons/Multimedia/HW/project/dat...  down
2  /mnt/g/onlinelessons/Multimedia/HW/project/dat...  down
3  /mnt/g/onlinelessons/Multimedia/HW/project/dat...  down
4  /mnt/g/onlinelessons/Multimedia/HW/project/dat...  down


In [14]:
train_df_encoded = pd.get_dummies(train_df.label, prefix='direction')
test_df_encoded = pd.get_dummies(test_df.label, prefix='direction')
val_df_encoded = pd.get_dummies(val_df.label, prefix='direction')

print(train_df_encoded.head())

   direction_down  direction_left  direction_right  direction_up
0               1               0                0             0
1               1               0                0             0
2               1               0                0             0
3               1               0                0             0
4               1               0                0             0


In [15]:
concated = [train_df, train_df_encoded]
train_df_encoded = pd.concat(concated, axis=1,names =[] , ignore_index=True, join='inner')

concated = [test_df, test_df_encoded]
test_df_encoded = pd.concat(concated, axis=1,names =[] , ignore_index=True, join='inner')

concated = [val_df, val_df_encoded]
val_df_encoded = pd.concat(concated, axis=1,names =[] , ignore_index=True, join='inner')

In [16]:
columns = {0 : "data", 1 :"label", 2: "direction_down", 3: "direction_left",
           4: "direction_right", 5: "direction_up"}    
train_df_encoded = train_df_encoded.rename(columns=columns)
test_df_encoded = test_df_encoded.rename(columns=columns)
val_df_encoded = val_df_encoded.rename(columns=columns)

In [17]:
print(val_df_encoded.head())

                                                data label  direction_down  \
0  /mnt/g/onlinelessons/Multimedia/HW/project/dat...  down               1   
1  /mnt/g/onlinelessons/Multimedia/HW/project/dat...  down               1   
2  /mnt/g/onlinelessons/Multimedia/HW/project/dat...  down               1   
3  /mnt/g/onlinelessons/Multimedia/HW/project/dat...  down               1   
4  /mnt/g/onlinelessons/Multimedia/HW/project/dat...  down               1   

   direction_left  direction_right  direction_up  
0               0                0             0  
1               0                0             0  
2               0                0             0  
3               0                0             0  
4               0                0             0  


In [18]:
train_df_encoded.to_csv("./dataset/final_data/train_data_encoded.csv")
test_df_encoded.to_csv("./dataset/final_data/test_data_encoded.csv")
val_df_encoded.to_csv("./dataset/final_data/val_data_encoded.csv")